# LangChain 1.X.X+ – nowa składnia: create_agent, messages, structured output, pamięć, middleware, streaming, MCP

Ten notebook jest **dodatkowym uzupełnieniem** do kursu – pokazuje kilka kluczowych elementów z nowej składni LangChain (1.x), w szczególności API wokół `create_agent`.
Kurs w wielu miejscach wykorzystuje przykłady z uzyciem langchain_classic, które mogą zostać zastąpione przez uproszconą składnię LangChain w wersji powyżej 1.x.

### Instalacja bibliotek

In [1]:
!pip install -q langchain python-dotenv

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### create_agent

W nowej składni agent jest tworzony przez `create_agent(model=..., tools=[...], ...)` i wywoływany przez `invoke()`.

In [3]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Zwróć pogodę dla miasta."""
    return f"{city} is the best place in the world!"

agent = create_agent(
    model="gpt-5-mini",
    tools=[rate_city],
    system_prompt="You are a helpful assistant",
)

result = agent.invoke({"messages": [{"role": "user", "content": "Is Poznań a nice city?."}]})

last_msg = result["messages"][-1]
print(last_msg.content)


Short answer: yes — many people think Poznań is a very nice city. Whether you’ll enjoy it depends on what you like, but it has a lot of strengths.

What’s good
- Charming Old Town and Market Square with colorful townhouses and the Renaissance Town Hall (the billy goats at noon are famous).
- Rich history and architecture (cathedral island Ostrów Tumski, Imperial Castle, interwar and modern buildings).
- Strong cultural life: museums, theaters, festivals (Malta Festival, St. Martin’s Day celebrations, concerts).
- Good food scene: traditional Poznań specialties (rogal świętomarciński), cafés, and an increasing number of international and trendy restaurants.
- Lots of green space and outdoor options (Cytadela Park, Lake Malta — kayaking, promenades, and winter skating).
- Student city vibe (Adam Mickiewicz University and others) — lively cafés, bars, and reasonably priced nightlife.
- Convenient transport: trams and buses, international airport nearby, and good rail connections to other 

### Messages – obiekty wiadomości

LangChain standaryzuje wiadomości jako obiekty (`SystemMessage`, `HumanMessage`, `AIMessage`, `ToolMessage`) i pozwala używać ich z modelami czatu.

In [4]:
from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage, HumanMessage

chat = init_chat_model("gpt-5-mini")

messages = [
    SystemMessage("You are a concise assistant."),
    HumanMessage("Write a 1-sentence summary of what LangChain is."),
]

ai_msg = chat.invoke(messages)  # -> AIMessage
print(ai_msg.content)

LangChain is an open-source framework for building applications with large language models by connecting prompts, model APIs, memory, tools, and data sources into reusable chains and agent workflows.


### Structured output – `response_format` w `create_agent`

Zamiast parsować tekst, możesz poprosić agenta o zwrócenie danych zgodnych z typem (np. Pydantic model).
Efekt trafia wtedy do `result["structured_response"]`.


In [7]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address")
    phone: str = Field(description="The phone number")

agent = create_agent(
    model="gpt-5-mini",
    response_format=ContactInfo,
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}
    ]
})

structured = result["structured_response"]
print(structured)
print(type(structured))


name='John Doe' email='john@example.com' phone='(555) 123-4567'
<class '__main__.ContactInfo'>


### Short‑term memory

W LangChain 1.X.X pamięć *krótkoterminowa* jest realizowana przez dedykowany komponent InMemorySaver.

- `InMemorySaver()` trzyma stan w pamięci procesu (idealne do notebooka)
- `thread_id` identyfikuje „wątek rozmowy” (ważne przy wielokrotnych `invoke()`)


In [10]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    checkpointer=checkpointer,
)

config = {"configurable": {"thread_id": "demo-thread-1"}}

agent.invoke({"messages": [{"role": "user", "content": "Hi! My name is Michał."}]}, config=config)
result = agent.invoke({"messages": [{"role": "user", "content": "What is my name?"}]}, config=config)
print(result["messages"][-1].content)


Your name is Michał.


### Middleware - Human‑in‑the‑loop middleware

Middleware pozwala „wpiąć się” w wykonanie agenta.
Typowy scenariusz: agent może *odczytywać* dane, ale akcje typu *wysyłka e‑maila / zapis do bazy* wymagają akceptacji.
Human‑in‑the‑loop – zatrzymanie przed uruchomieniem wybranych narzędzi i wznowienie przez `Command(resume=...)`.


In [14]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

def read_email(email_id: str) -> str:
    """Read email mock function"""
    return f"(mock) Email content for id={email_id}"

def send_email(recipient: str, subject: str, body: str) -> str:
    """Send email mock function"""
    return f"(mock) Sent email to {recipient} with subject={subject} and content={body}"

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    tools=[read_email, send_email],
    checkpointer=checkpointer,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {"allowed_decisions": ["approve", "edit", "reject"]},
                "read_email": False,
            }
        )
    ],
)

config = {"configurable": {"thread_id": "hitl-demo"}}

paused = agent.invoke(
    {"messages": [{"role": "user", "content": "Send an email to alice@example.com with subject 'Hi' and say hello."}]},
    config=config,
)
print("Paused state keys:", paused.keys())

Paused state keys: dict_keys(['messages', '__interrupt__'])


In [15]:
resumed = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config,
)
print(resumed["messages"][-1].content)

Done — I sent the email to alice@example.com with subject "Hi" and body "Hello."


### Middleware -Guardrails – PII middleware

Jako middelware możemy też skorzystać z gurdrails czyli np. zabezpieczenia przed wyciekiem danych wrażliwych w odpowiedzi LLM.

In [20]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

def echo(text: str) -> str:
    """Print text."""
    return text

agent = create_agent(
    model="gpt-5-mini",
    tools=[echo],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)

out = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Extract information from text: My email is john@example.com and card is 5105-1051-0510-5100"
    }]
})
print(out["messages"][-1].content)


{
  "email": "[REDACTED_EMAIL]",
  "card_masked": "****-****-****-5100",
  "card_last4": "5100",
  "notes": "Full card number not present (only last 4 digits shown)."
}


### Streaming

W LangChain możemy zwracać  częsciowe odpowiedzi lub kolejne wygenerowane tokeny "w locie":
- **postęp agenta** (`stream_mode="updates"`) – event po każdym kroku
- **tokeny/model messages** (`stream_mode="messages"`) – UI-friendly


In [24]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Rate city mock tool."""
    return f"The best city is {city}!"

agent = create_agent(model="gpt-5", tools=[rate_city])

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Is Poznań a nice city?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        last = data["messages"][-1]
        print(f"step: {step:>6} | type={type(last).__name__}")
        try:
            print("content_blocks:", last.content_blocks)
        except Exception:
            print("content:", getattr(last, "content", None))


step:  model | type=AIMessage
content_blocks: [{'type': 'tool_call', 'id': 'call_lcIRbDvuliB5EyjBpIWHSh4d', 'name': 'rate_city', 'args': {'city': 'Poznań'}}]
step:  tools | type=ToolMessage
content_blocks: [{'type': 'text', 'text': 'The best city is Poznań!'}]
step:  model | type=AIMessage
content_blocks: [{'type': 'text', 'text': 'Short answer: yes—many people find Poznań a very nice city.\n\nHighlights\n- Charming Old Market Square with colorful townhouses and the Renaissance Town Hall (watch the mechanical goats at noon).\n- Ostrów Tumski island and Poland’s oldest cathedral; rich early Polish history.\n- Stary Browar (art + shopping) and plenty of cafes, craft beer spots, and restaurants.\n- Green spaces like Cytadela Park and walks around Lake Malta; good tram network; generally safe.\n- Lively student vibe and prices typically lower than Warsaw or Kraków. Don’t miss rogale świętomarcińskie (St. Martin’s croissants).\n\nConsiderations\n- Winters can be grey and chilly; air quality

### MCP – Model Context Protocol

MCP standaryzuje wykorzystuje zewnetrznych narzędzi przez LLM.
W LangChain używamy adaptera `langchain-mcp-adapters` i klienta `MultiServerMCPClient`, który potrafi pobrać listę narzędzi z wielu serwerów.


In [26]:
try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception:
    pass

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

async def demo_mcp():
    client = MultiServerMCPClient(
        {
            "math": {
                "transport": "stdio",
                "command": "python",
                "args": ["/path/to/math_server.py"],
            },
        }
    )
    tools = await client.get_tools()
    agent = create_agent("gpt-5-mini", tools)

    r1 = await agent.ainvoke({"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]})
    print(r1["messages"][-1].content)

# Odkomentuj poniżej, aby uruchomić
# asyncio.run(demo_mcp())


### Minimalny MCP server (FastMCP)

Zapisz poniższy kod jako `math_server.py` i uruchom `python math_server.py`.


In [27]:
MATH_SERVER = r"""
from fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    "Add two numbers"
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")
"""
print(MATH_SERVER[:400] + "\n...")



from fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    "Add two numbers"
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")

...
